# Beer Analysis



In [2]:
# Import Dependencies
import pandas as pd
from pathlib import Path
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

In [3]:
# Path to the cleaned .csv file
path = "Outputs/cleanbeer.csv"

# Read file into a Pandas dataframe
beer_df = pd.read_csv(path)

# Preview file
beer_df.head()

,Beer Name (Full),Name,Brewery,Style_Main,Style_Sub,ABV,Min IBU,Max IBU,Astringency,Body,...,Fruits,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall,number_of_reviews
0,Alaskan Brewing Co. Alaskan Amber,Amber,Alaskan Brewing Co.,Altbier,NaN,5.3,25,50,13,32,...,33,57,8,111,3.498994,3.636821,3.556338,3.643863,3.847082,497
1,Long Trail Brewing Co. Double Bag,Double Bag,Long Trail Brewing Co.,Altbier,NaN,7.2,25,50,12,57,...,24,35,12,84,3.798337,3.846154,3.904366,4.024948,4.034304,481
2,Long Trail Brewing Co. Long Trail Ale,Long Trail Ale,Long Trail Brewing Co.,Altbier,NaN,5.0,25,50,14,37,...,10,54,4,62,3.409814,3.667109,3.600796,3.631300,3.830239,377
3,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...,Doppelsticke,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,Altbier,NaN,8.5,25,50,13,55,...,49,40,16,119,4.148098,4.033967,4.150815,4.205163,4.005435,368
4,Ninkasi Brewing Company Sleigh'r Dark Doüble A...,Sleigh'r Dark Doüble Alt Ale,Ninkasi Brewing Company,Altbier,NaN,7.2,25,50,25,51,...,11,51,20,95,3.625000,3.973958,3.734375,3.765625,3.817708,96


# Question 1: 

#### lorem ipsum

In [ ]:
# answer/plot here



# Question 2: 

#### lorem ipsum

In [ ]:
# answer/plot here



# Question 3: 

#### lorem ipsum

In [ ]:
# answer/plot here

